<a href="https://colab.research.google.com/github/woochichi/opencv_image-preprocessing-noise/blob/main/remove_noise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>




Non-local means Denoising

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('dark_background')

In [ ]:
img_ori = cv2.imread('sample (6).jpg')
img_copy = img_ori.copy()

height, width, channel = img_ori.shape

plt.figure(figsize=(12, 10))
plt.imshow(img_ori, cmap='gray')

In [ ]:
#gray 변환
img_gray = cv2.cvtColor(img_ori,cv2.COLOR_BGR2GRAY)
#img_gray = ~img_gray

plt.figure(figsize=(12, 10))
plt.imshow(img_gray, cmap='gray')

In [ ]:
# Non-local Means  잡음 제거
# fastNlMeansDenoising: 단일 gray image에 작동
# 1) h: 필터 강도 -> 높은 값일 수록 잡음 더 잘 제거
img_Denoising = cv2.fastNlMeansDenoising(img_gray, h = 20, templateWindowSize=7, searchWindowSize=21)

plt.figure(figsize=(12, 10))
plt.imshow(img_Denoising, cmap='gray')

In [ ]:
# GuassianBlur
img_Denoising = cv2.GaussianBlur(img_Denoising, ksize=(5, 5), sigmaX=0)
plt.figure(figsize=(12, 10))
plt.imshow(img_Denoising, cmap='gray')

threshold + countour

In [ ]:
ret, img_Denoising = cv2.threshold(img_Denoising, 180, 255, cv2.THRESH_BINARY_INV)
plt.figure(figsize=(12, 10))
plt.imshow(img_Denoising, cmap='gray')

img_thresh = cv2.adaptiveThreshold(
    img_Denoising, 
    maxValue=255.0, 
    adaptiveMethod=cv2.ADAPTIVE_THRESH_GAUSSIAN_C, 
    thresholdType=cv2.THRESH_BINARY_INV, 
    blockSize=19, 
    C=9
)

plt.figure(figsize=(12, 10))
plt.imshow(img_thresh, cmap='gray')

In [ ]:
contours, _ = cv2.findContours(
    img_thresh, 
    mode=cv2.RETR_LIST, 
    method=cv2.CHAIN_APPROX_SIMPLE
)

temp_result = np.zeros((height, width, channel), dtype=np.uint8)

cv2.drawContours(temp_result, contours=contours, contourIdx=-1, color=(255, 255, 255))

plt.figure(figsize=(12, 10))
plt.imshow(temp_result)

detecting test

In [ ]:
contours, _ = cv2.findContours(
    img_thresh, 
    mode=cv2.RETR_LIST, 
    method=cv2.CHAIN_APPROX_SIMPLE
)

temp_result = np.zeros((height, width, channel), dtype=np.uint8)

contours_dict = []

for contour in contours:
    x, y, w, h = cv2.boundingRect(contour)
    cv2.rectangle(temp_result, pt1=(x, y), pt2=(x+w, y+h), color=(255, 255, 255), thickness=2)
    
    # insert to dict
    contours_dict.append({
        'contour': contour,
        'x': x,
        'y': y,
        'w': w,
        'h': h,
        'cx': x + (w / 2),
        'cy': y + (h / 2)
    })

plt.figure(figsize=(12, 10))
plt.imshow(temp_result, cmap='gray')

In [ ]:
MIN_AREA = 80
MIN_WIDTH, MIN_HEIGHT = 2, 8
MIN_RATIO, MAX_RATIO = 0.25, 1.0

possible_contours = []

cnt = 0
for d in contours_dict:
    area = d['w'] * d['h']
    ratio = d['w'] / d['h']
    
    if area > MIN_AREA \
    and d['w'] > MIN_WIDTH and d['h'] > MIN_HEIGHT \
    and MIN_RATIO < ratio < MAX_RATIO:
        d['idx'] = cnt
        cnt += 1
        possible_contours.append(d)
        
# visualize possible contours
temp_result = np.zeros((height, width, channel), dtype=np.uint8)

for d in possible_contours:
#     cv2.drawContours(temp_result, d['contour'], -1, (255, 255, 255))
    cv2.rectangle(temp_result, pt1=(d['x'], d['y']), pt2=(d['x']+d['w'], d['y']+d['h']), color=(255, 255, 255), thickness=2)

plt.figure(figsize=(12, 10))
plt.imshow(temp_result, cmap='gray')

In [ ]:
MAX_DIAG_MULTIPLYER = 5 # 5
MAX_ANGLE_DIFF = 12.0 # 12.0
MAX_AREA_DIFF = 0.5 # 0.5
MAX_WIDTH_DIFF = 0.8
MAX_HEIGHT_DIFF = 0.2
MIN_N_MATCHED = 3 # 3

def find_chars(contour_list):
    matched_result_idx = []
    
    for d1 in contour_list:
        matched_contours_idx = []
        for d2 in contour_list:
            if d1['idx'] == d2['idx']:
                continue

            dx = abs(d1['cx'] - d2['cx'])
            dy = abs(d1['cy'] - d2['cy'])

            diagonal_length1 = np.sqrt(d1['w'] ** 2 + d1['h'] ** 2)

            distance = np.linalg.norm(np.array([d1['cx'], d1['cy']]) - np.array([d2['cx'], d2['cy']]))
            if dx == 0:
                angle_diff = 90
            else:
                angle_diff = np.degrees(np.arctan(dy / dx))
            area_diff = abs(d1['w'] * d1['h'] - d2['w'] * d2['h']) / (d1['w'] * d1['h'])
            width_diff = abs(d1['w'] - d2['w']) / d1['w']
            height_diff = abs(d1['h'] - d2['h']) / d1['h']

            if distance < diagonal_length1 * MAX_DIAG_MULTIPLYER \
            and angle_diff < MAX_ANGLE_DIFF and area_diff < MAX_AREA_DIFF \
            and width_diff < MAX_WIDTH_DIFF and height_diff < MAX_HEIGHT_DIFF:
                matched_contours_idx.append(d2['idx'])

        # append this contour
        matched_contours_idx.append(d1['idx'])

        if len(matched_contours_idx) < MIN_N_MATCHED:
            continue

        matched_result_idx.append(matched_contours_idx)

        unmatched_contour_idx = []
        for d4 in contour_list:
            if d4['idx'] not in matched_contours_idx:
                unmatched_contour_idx.append(d4['idx'])

        unmatched_contour = np.take(possible_contours, unmatched_contour_idx)
        
        # recursive
        recursive_contour_list = find_chars(unmatched_contour)
        
        for idx in recursive_contour_list:
            matched_result_idx.append(idx)

        break

    return matched_result_idx
    
result_idx = find_chars(possible_contours)

matched_result = []
for idx_list in result_idx:
    matched_result.append(np.take(possible_contours, idx_list))

# visualize possible contours
temp_result = np.zeros((height, width, channel), dtype=np.uint8)

for r in matched_result:
    for d in r:
#         cv2.drawContours(temp_result, d['contour'], -1, (255, 255, 255))
        cv2.rectangle(temp_result, pt1=(d['x'], d['y']), pt2=(d['x']+d['w'], d['y']+d['h']), color=(255, 255, 255), thickness=2)

plt.figure(figsize=(12, 10))
plt.imshow(temp_result, cmap='gray')

In [ ]:
PLATE_WIDTH_PADDING = 1.3 # 1.3
PLATE_HEIGHT_PADDING = 1.5 # 1.5
MIN_PLATE_RATIO = 3
MAX_PLATE_RATIO = 10

plate_imgs = []
plate_infos = []

for i, matched_chars in enumerate(matched_result):
    sorted_chars = sorted(matched_chars, key=lambda x: x['cx'])

    plate_cx = (sorted_chars[0]['cx'] + sorted_chars[-1]['cx']) / 2
    plate_cy = (sorted_chars[0]['cy'] + sorted_chars[-1]['cy']) / 2
    
    plate_width = (sorted_chars[-1]['x'] + sorted_chars[-1]['w'] - sorted_chars[0]['x']) * PLATE_WIDTH_PADDING
    
    sum_height = 0
    for d in sorted_chars:
        sum_height += d['h']

    plate_height = int(sum_height / len(sorted_chars) * PLATE_HEIGHT_PADDING)
    
    triangle_height = sorted_chars[-1]['cy'] - sorted_chars[0]['cy']
    triangle_hypotenus = np.linalg.norm(
        np.array([sorted_chars[0]['cx'], sorted_chars[0]['cy']]) - 
        np.array([sorted_chars[-1]['cx'], sorted_chars[-1]['cy']])
    )
    
    #angle = np.degrees(np.arcsin(triangle_height / triangle_hypotenus))
    
    #rotation_matrix = cv2.getRotationMatrix2D(center=(plate_cx, plate_cy), angle=angle, scale=1.0)
    
    #img_rotated = cv2.warpAffine(img_thresh, M=rotation_matrix, dsize=(width, height))
    
    img_cropped = cv2.getRectSubPix(
        img_thresh, 
        patchSize=(int(plate_width), int(plate_height)), 
        center=(int(plate_cx), int(plate_cy))
    )
    
    if img_cropped.shape[1] / img_cropped.shape[0] < MIN_PLATE_RATIO or img_cropped.shape[1] / img_cropped.shape[0] < MIN_PLATE_RATIO > MAX_PLATE_RATIO:
        continue
    
    plate_imgs.append(img_cropped)
    plate_infos.append({
        'x': int(plate_cx - plate_width / 2),
        'y': int(plate_cy - plate_height / 2),
        'w': int(plate_width),
        'h': int(plate_height)
    })
    
    plt.subplot(len(matched_result), 1, i+1)
    plt.imshow(img_cropped, cmap='gray')

In [ ]:
longest_idx, longest_text = -1, 0
plate_chars = []

for i, plate_img in enumerate(plate_imgs):
    plate_img = cv2.resize(plate_img, dsize=(0, 0), fx=1.6, fy=1.6)
    _, plate_img = cv2.threshold(plate_img, thresh=0.0, maxval=255.0, type=cv2.THRESH_BINARY | cv2.THRESH_OTSU)
    
    # find contours again (same as above)
    contours, _ = cv2.findContours(plate_img, mode=cv2.RETR_LIST, method=cv2.CHAIN_APPROX_SIMPLE)
    
    plate_min_x, plate_min_y = plate_img.shape[1], plate_img.shape[0]
    plate_max_x, plate_max_y = 0, 0

    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)
        
        area = w * h
        ratio = w / h

        if area > MIN_AREA \
        and w > MIN_WIDTH and h > MIN_HEIGHT \
        and MIN_RATIO < ratio < MAX_RATIO:
            if x < plate_min_x:
                plate_min_x = x
            if y < plate_min_y:
                plate_min_y = y
            if x + w > plate_max_x:
                plate_max_x = x + w
            if y + h > plate_max_y:
                plate_max_y = y + h
                
    img_result = plate_img[plate_min_y:plate_max_y, plate_min_x:plate_max_x]
    
    img_result = cv2.GaussianBlur(img_result, ksize=(3, 3), sigmaX=0)
    _, img_result = cv2.threshold(img_result, thresh=0.0, maxval=255.0, type=cv2.THRESH_BINARY | cv2.THRESH_OTSU)
    img_result = cv2.copyMakeBorder(img_result, top=10, bottom=10, left=10, right=10, borderType=cv2.BORDER_CONSTANT, value=(0,0,0))

    #chars = pytesseract.image_to_string(img_result, lang='kor', config='--psm 7 --oem 0')
    
    result_chars = ''
    has_digit = False
    '''
    for c in chars:
        if ord('가') <= ord(c) <= ord('힣') or c.isdigit():
            if c.isdigit():
                has_digit = True
            result_chars += c
    '''
    print(result_chars)
    plate_chars.append(result_chars)

    if has_digit and len(result_chars) > longest_text:
        longest_idx = i

    plt.subplot(len(plate_imgs), 1, i+1)
    plt.imshow(img_result, cmap='gray')

In [ ]:
info = plate_infos[longest_idx]
chars = plate_chars[longest_idx]

print(chars)

img_out = img_ori.copy()

cv2.rectangle(img_out, pt1=(info['x'], info['y']), pt2=(info['x']+info['w'], info['y']+info['h']), color=(255,0,0), thickness=2)

cv2.imwrite(chars + '.jpg', img_out)

plt.figure(figsize=(12, 10))
plt.imshow(img_out)

이미지 기울기 조정 - Deskewing

In [ ]:
contours = sorted(contours, key = cv2.contourArea, reverse = True)
largestContour = contours[0]
minAreaRect = cv2.minAreaRect(largestContour)

box = cv2.boxPoints(minAreaRect)
box = np.int0(box)
cv2.drawContours(temp_result, [box], 0, (0, 255, 0), 3)

print(box)

plt.figure(figsize=(12, 10))
plt.imshow(temp_result)

angle = minAreaRect[-1]
if angle < -45:
    angle = 90 + angle
angle = (-1.0) * angle

print(angle, "angle")

center = (width/2, height/2)
M = cv2.getRotationMatrix2D(center, angle * (-1.0), 1.0)
img_thresh = cv2.warpAffine(img_thresh, M, (width,height), flags=cv2.INTER_CUBIC, borderMode=cv2.BORDER_REPLICATE)

left_down = box[1]
right_down = box[0]
left_up = box[2]
right_up = box[3]
src = np.float32([left_down, right_down, left_up, right_up])
dst = np.float32([[0, 0], [int(max(minAreaRect[1][0], minAreaRect[1][1])), 0], [0, int(min(minAreaRect[1][0], minAreaRect[1][1]))],
                              [int(max(minAreaRect[1][0], minAreaRect[1][1])),
                               int(min(minAreaRect[1][0], minAreaRect[1][1]))]])
m = cv2.getPerspectiveTransform(src, dst)
img_warp = cv2.warpPerspective(img_copy, m, (int(max(minAreaRect[1][0], minAreaRect[1][1])), int(min(minAreaRect[1][0], minAreaRect[1][1]))),
                                         flags=cv2.INTER_CUBIC)
'''
edges = cv2.Canny(temp_result, 100, 200)
lines = cv2.HoughLines(edges, 1, np.pi/180, 130)
for line in lines:
    r,theta = line[0]
    tx, ty = np.cos(theta), np.sin(theta)
    x0, y0 = tx*r, ty*r

    cv2.circle(img_copy, (abs(x0), abs(y0)), 3, (0,0,255), -1)

    x1, y1 = int(x0 + w*(-ty)), int(y0 + h*tx)
    x2, y2 = int(x0 - w *(-ty)), int(y0 - h * tx)

    cv2.line(img_copy, (x1, y1), (x2, y2), (0,255,0), 1)
'''


plt.figure(figsize=(12, 10))
plt.imshow(img_thresh)

img_crop = cv2.warpAffine(img_copy, M, (width,height), flags=cv2.INTER_CUBIC, borderMode=cv2.BORDER_REPLICATE)
plt.figure(figsize=(12, 10))
plt.imshow(img_crop)

cv2.drawContours(img_copy, [box], 0, (0, 255, 0), 3)
plt.figure(figsize=(12, 10))
plt.imshow(img_copy)

plt.figure(figsize=(12, 10))
plt.imshow(img_warp)

Gaussian Blur

In [ ]:
img_blurred = cv2.GaussianBlur(img_gray, ksize=(5, 5), sigmaX=0)
plt.figure(figsize=(12, 10))
plt.imshow(img_blurred, cmap='gray')

ret, img_blurred = cv2.threshold(img_blurred, 200, 255, cv2.THRESH_BINARY)


img_thresh = cv2.adaptiveThreshold(
    img_blurred, 
    maxValue=255.0, 
    adaptiveMethod=cv2.ADAPTIVE_THRESH_GAUSSIAN_C, 
    thresholdType=cv2.THRESH_BINARY_INV, 
    blockSize=19, 
    C=9
)

plt.figure(figsize=(12, 10))
plt.imshow(img_thresh, cmap='gray')

In [ ]:
contours, _ = cv2.findContours(
    img_thresh, 
    mode=cv2.RETR_LIST, 
    method=cv2.CHAIN_APPROX_SIMPLE
)

temp_result = np.zeros((height, width, channel), dtype=np.uint8)

cv2.drawContours(temp_result, contours=contours, contourIdx=-1, color=(255, 255, 255))

plt.figure(figsize=(12, 10))
plt.imshow(temp_result)

In [ ]:
test_img = cv2.imread('original_sample_1.jpg')
img_copy = test_img.copy()

height, width, channel = test_img.shape

plt.figure(figsize=(12, 10))
plt.imshow(test_img, cmap='gray')

In [ ]:
#gray 변환
test_img_gray = cv2.cvtColor(test_img,cv2.COLOR_BGR2GRAY)

plt.figure(figsize=(12, 10))
plt.imshow(test_img_gray, cmap='gray')

In [ ]:
test_bilateral = cv2.bilateralFilter(test_img_gray,9,75,75)

plt.figure(figsize=(12, 10))
plt.imshow(test_bilateral, cmap='gray')

In [ ]:
test_median = cv2.medianBlur(test_img_gray,5)

plt.figure(figsize=(12, 10))
plt.imshow(test_median, cmap='gray')

In [ ]:
# Non-local Means  잡음 제거
# fastNlMeansDenoising: 단일 gray image에 작동
# 1) h: 필터 강도 -> 높은 값일 수록 잡음 더 잘 제거
img_Denoising = cv2.fastNlMeansDenoising(img_gray, h = 20, templateWindowSize=7, searchWindowSize=21)

plt.figure(figsize=(12, 10))
plt.imshow(img_Denoising, cmap='gray')

In [ ]:
ret1, img_Denoising = cv2.threshold(img_Denoising, 200, 255, cv2.THRESH_BINARY)
ret2, test_bilateral = cv2.threshold(test_bilateral, 200, 255, cv2.THRESH_BINARY)
ret3, test_median = cv2.threshold(test_median, 200, 255, cv2.THRESH_BINARY)


img_thresh = cv2.adaptiveThreshold(
    img_Denoising, 
    maxValue=255.0, 
    adaptiveMethod=cv2.ADAPTIVE_THRESH_GAUSSIAN_C, 
    thresholdType=cv2.THRESH_BINARY_INV, 
    blockSize=19, 
    C=9
)

img_thresh2 = cv2.adaptiveThreshold(
    test_bilateral, 
    maxValue=255.0, 
    adaptiveMethod=cv2.ADAPTIVE_THRESH_GAUSSIAN_C, 
    thresholdType=cv2.THRESH_BINARY_INV, 
    blockSize=19, 
    C=9
)

img_thresh3 = cv2.adaptiveThreshold(
    test_median, 
    maxValue=255.0, 
    adaptiveMethod=cv2.ADAPTIVE_THRESH_GAUSSIAN_C, 
    thresholdType=cv2.THRESH_BINARY_INV, 
    blockSize=19, 
    C=9
)

plt.figure(figsize=(12, 10))
plt.imshow(img_thresh, cmap='gray')
plt.figure(figsize=(12, 10))
plt.imshow(img_thresh2, cmap='gray')
plt.figure(figsize=(12, 10))
plt.imshow(img_thresh3, cmap='gray')

In [ ]:
contours, _ = cv2.findContours(
    img_thresh, 
    mode=cv2.RETR_LIST, 
    method=cv2.CHAIN_APPROX_SIMPLE
)
contours2, _ = cv2.findContours(
    img_thresh2, 
    mode=cv2.RETR_LIST, 
    method=cv2.CHAIN_APPROX_SIMPLE
)
contours3, _ = cv2.findContours(
    img_thresh3, 
    mode=cv2.RETR_LIST, 
    method=cv2.CHAIN_APPROX_SIMPLE
)


temp_result = np.zeros((height, width, channel), dtype=np.uint8)
temp_result2 = np.zeros((height, width, channel), dtype=np.uint8)
temp_result3 = np.zeros((height, width, channel), dtype=np.uint8)

cv2.drawContours(temp_result, contours=contours, contourIdx=-1, color=(255, 255, 255))
cv2.drawContours(temp_result2, contours=contours2, contourIdx=-1, color=(255, 255, 255))
cv2.drawContours(temp_result3, contours=contours3, contourIdx=-1, color=(255, 255, 255))

plt.figure(figsize=(12, 10))
plt.imshow(temp_result)
plt.figure(figsize=(12, 10))
plt.imshow(temp_result2)
plt.figure(figsize=(12, 10))
plt.imshow(temp_result3)